### **1. Installing the pyspark and Py4j module.**

In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 57 kB/s 
     |████████████████████████████████| 200 kB 82.6 MB/s 
     |████████████████████████████████| 199 kB 96.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=4f78f8f446dce427984cb1f491a3257db0747326fb7903831c3bd0d1009fa04e
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName("Decision Tree").getOrCreate()

In [ ]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Big Data and IoT Files/Lab Contents/Lab 8: Decision Tree/Iris.csv", inferSchema = True, header = True)

In [ ]:
df.show()

In [ ]:
df.count()

In [ ]:
len(df.columns)

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
df.head(5)

In [ ]:
df.printSchema()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol = "Species", outputCol = "Species_cat")

In [ ]:
indexed = indexer.fit(df).transform(df)

In [ ]:
indexed.show()

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"], outputCol = "features")

In [ ]:
assembler

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.show(truncate = False)

In [ ]:
output.select("features","Species_cat").show(truncate = False)

In [ ]:
final_data = output.select("features","Species_cat")

In [ ]:
final_data.show()

In [ ]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
train_data.show()

In [ ]:
test_data.show()

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
df_classifier = DecisionTreeClassifier(labelCol = "Species_cat").fit(train_data)

In [ ]:
df_predictions = df_classifier.transform(test_data)

In [ ]:
df_predictions.show()

In [ ]:
df_accuracy = MulticlassClassificationEvaluator(labelCol= "Species_cat",
                                                metricName="accuracy").evaluate(df_predictions)

In [ ]:
df_accuracy

In [ ]:
df_precision = MulticlassClassificationEvaluator(labelCol= "Species_cat",
                                                metricName="weightedPrecision").evaluate(df_predictions)

In [ ]:
df_precision